In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
#from sklearn.cross_validation import train_test_split
# Building a english to french translator

In [ ]:
lines= pd.read_table('hin.txt', names=['eng', 'hin'])

In [ ]:
lines = lines[0:50000]

In [ ]:
lines.shape

(2810, 2)

In [ ]:
lines.sample(10)

,eng,hin
800,He betrayed his country.,उसने अपने देश को धोखा दे दिया।
944,Let me introduce my wife.,यह मेरी पत्नी है।
2639,My mother is constantly forgetting people's na...,मेरी माँ अक्सर लोगों के नाम भूलती रहती है।
630,I am grateful to them.,मैं उनका आभारी हूँ।
418,The batter was out.,बॅटर आऊट हो गया था।
348,What's the matter?,मुश्किल क्या है?
1428,He spread butter on the bread.,उसने मक्खन को ब्रेड पर लगाया।
905,I can't stand it anymore.,मुझसे और सहन नहीं होगा!
1065,One of my bags is missing.,मुझे अपनी एक अटैची मिल नहीं रही है।
1013,He's sleeping like a baby.,वह बच्चे की तरह सो रहा है।


In [ ]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

In [ ]:
# Take the length as 50
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [ ]:
exclude = set(string.punctuation)
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin=lines.hin.apply(lambda x: x.translate(remove_digits))

In [ ]:
lines.head()

,eng,hin
0,wow,वाह
1,help,बचाओ
2,jump,उछलो
3,jump,कूदो
4,jump,छलांग


In [ ]:
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.hin.head()

0      START_ वाह _END
1     START_ बचाओ _END
2     START_ उछलो _END
3     START_ कूदो _END
4    START_ छलांग _END
Name: hin, dtype: object

In [ ]:
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)


In [ ]:
len(all_eng_words), len(all_hindi_words)

(2359, 3011)

In [ ]:
lenght_list=[]
for l in lines.hin:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

29

In [ ]:
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
np.max(lenght_list)

23

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
# del all_eng_words, all_french_words

In [ ]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [ ]:
encoder_input_data = np.zeros(
    (len(lines.eng), 23),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.hin), 29),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.hin), 29, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(lines.eng, lines.hin)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.


In [ ]:
#Build keras encoder-decoder model

In [ ]:
embedding_size = 100

In [ ]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

In [ ]:
#Encoder model

In [ ]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
#Decoder model

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    235900      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    301100      input_6[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LS

In [ ]:
#Fit the model

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=500,
          validation_split=0.05)

Train on 2669 samples, validate on 141 samples
Epoch 1/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.5591 - acc: 0.0352 - val_loss: 3.0392 - val_acc: 0.0345
Epoch 2/500
2669/2669 [==============================] - 12s 5ms/step - loss: 1.5292 - acc: 0.0345 - val_loss: 3.0469 - val_acc: 0.0345
Epoch 3/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.5083 - acc: 0.0345 - val_loss: 3.0488 - val_acc: 0.0345
Epoch 4/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.4898 - acc: 0.0345 - val_loss: 3.0529 - val_acc: 0.0345
Epoch 5/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.4722 - acc: 0.0345 - val_loss: 3.0454 - val_acc: 0.0345
Epoch 6/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.4562 - acc: 0.0345 - val_loss: 3.0473 - val_acc: 0.0345
Epoch 7/500
2669/2669 [==============================] - 11s 4ms/step - loss: 1.4423 - acc: 0.0345 - val_loss: 3.0523 - val_acc: 0.0345
E

2669/2669 [==============================] - 11s 4ms/step - loss: 0.0975 - acc: 0.2440 - val_loss: 4.2832 - val_acc: 0.0609
Epoch 432/500
2669/2669 [==============================] - 11s 4ms/step - loss: 0.0975 - acc: 0.2442 - val_loss: 4.2848 - val_acc: 0.0607
Epoch 433/500
2669/2669 [==============================] - 12s 4ms/step - loss: 0.0958 - acc: 0.2449 - val_loss: 4.2788 - val_acc: 0.0597
Epoch 434/500
2669/2669 [==============================] - 12s 4ms/step - loss: 0.0953 - acc: 0.2452 - val_loss: 4.2851 - val_acc: 0.0602
Epoch 435/500
2669/2669 [==============================] - 12s 4ms/step - loss: 0.0953 - acc: 0.2446 - val_loss: 4.2983 - val_acc: 0.0599
Epoch 436/500
2669/2669 [==============================] - 11s 4ms/step - loss: 0.0936 - acc: 0.2452 - val_loss: 4.2990 - val_acc: 0.0584
Epoch 437/500
2669/2669 [==============================] - 12s 4ms/step - loss: 0.0943 - acc: 0.2452 - val_loss: 4.3102 - val_acc: 0.0589
Epoch 438/500
2669/2669 [=======================

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 100)         235900    
_________________________________________________________________
lstm_3 (LSTM)                [(None, 50), (None, 50),  30200     
Total params: 266,100
Trainable params: 266,100
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Create sampling model

In [ ]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [ ]:
#Function to generate sequences

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
#Look at the some translations

In [ ]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 0    wow
Name: eng, dtype: object
Decoded sentence:  समुद्र तट कहां है _END
-
Input sentence: 1    help
Name: eng, dtype: object
Decoded sentence:  बचाओ _END
-
Input sentence: 2    jump
Name: eng, dtype: object
Decoded sentence:  छलांग _END
-
Input sentence: 3    jump
Name: eng, dtype: object
Decoded sentence:  छलांग _END
-
Input sentence: 4    jump
Name: eng, dtype: object
Decoded sentence:  छलांग _END
-
Input sentence: 5    hello
Name: eng, dtype: object
Decoded sentence:  समुद्र तट कहां है _END
-
Input sentence: 6    hello
Name: eng, dtype: object
Decoded sentence:  समुद्र तट कहां है _END
-
Input sentence: 7    cheers
Name: eng, dtype: object
Decoded sentence:  है कोटे मानो या न उत्तर कहाँ _END
-
Input sentence: 8    cheers
Name: eng, dtype: object
Decoded sentence:  है कोटे मानो या न उत्तर कहाँ _END
-
Input sentence: 9    got it
Name: eng, dtype: object
Decoded sentence:  समझे कि नहीं _END
-
Input sentence: 10    im ok
Name: eng, dtype: object
Decoded sentence:  म

-
Input sentence: 89    nobody came
Name: eng, dtype: object
Decoded sentence:  कोई नहीं आया। _END
-
Input sentence: 90    was i wrong
Name: eng, dtype: object
Decoded sentence:  क्या मैं ग़लत था _END
-
Input sentence: 91    whats this
Name: eng, dtype: object
Decoded sentence:  यह क्या है _END
-
Input sentence: 92    are you sick
Name: eng, dtype: object
Decoded sentence:  क्या तुम बीमार हो _END
-
Input sentence: 93    bring him in
Name: eng, dtype: object
Decoded sentence:  उसको अंदर ले आओ। _END
-
Input sentence: 94    come with us
Name: eng, dtype: object
Decoded sentence:  हमारे साथ आओ। _END
-
Input sentence: 95    happy easter
Name: eng, dtype: object
Decoded sentence:  एसटर मुबारक हो _END
-
Input sentence: 96    has tom left
Name: eng, dtype: object
Decoded sentence:  टॉम चला गया क्या _END
-
Input sentence: 97    he is french
Name: eng, dtype: object
Decoded sentence:  वह फ़्रानसीसी है। _END
-
Input sentence: 98    i am at home
Name: eng, dtype: object
Decoded sentence:  मैं घर प

In [ ]:
#As we can see above most of the sentences are being translated correctly 